In [46]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [36]:
model = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4", trainable=True)

In [ ]:
m = tf.keras.Sequential([
    model
])

m.build([None, 299, 299, 3])

m.summary()

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_key_points = np.array([[ 
        89.80914307, 105.43384552, 170.66313171,  84.76891327,
       110.0443573 , 172.53369141,  85.02732849, 103.94306183,
       136.14315796,  83.78937531, 124.95746613, 101.53977203,
        84.62630463, 107.87869263, 100.96289825,  88.19257355,
       144.24630737, 133.31724548, 100.9705658 , 107.33512878,
       103.92945099,  87.04039764, 168.51373291, 222.18237305,
       134.94984436, 103.54360962,  91.68307495,  70.75968933,
       156.53161621, 123.721138  , 134.4574585 , 114.10692596,
        18.21217155, 143.94229126, 156.27694702, 228.51443481,
       150.64054871, 128.60699463, 134.8057251 , 164.90638733,
       182.89833069, 177.72737122, 173.8009491 , 116.91728973,
        82.13097382, 194.67362976, 213.48432922, 176.45010376,
       190.55049133, 130.79293823,  94.19467926,  52.27914429,
        83.12620544, 208.3157959 , 232.10578918, 125.0796814 ]]).astype(np.float32)




In [67]:
pre_trained_model = InceptionV3(input_shape = (256, 256, 3),
                                include_top = False,
                                weights = 'imagenet')

image_path = 'test_images/take01.jpeg'
image = tf.io.read_file(image_path)
image = tf.compat.v1.image.decode_jpeg(image)
X = tf.expand_dims(image, axis=0)
X = tf.cast(tf.image.resize_with_pad(X, 256, 256), dtype=tf.int32)

pre_trained_model.trainable = False

m = tf.keras.Sequential([
    pre_trained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(56, activation='relu'),
])

m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

m.fit(X, train_key_points, epochs=1)





1/1 [==============================] - 8s 8s/step - loss: -290.4718 - accuracy: 0.0000e+00


In [87]:
points = m.predict(X)
print(points.shape)

label = ["nose", "left eye", "right eye", "left ear", "right ear", "left shoulder", "right shoulder", "left elbow", "right elbow", "left wrist", "right wrist", "left hip", "right hip", "left knee", "right knee", "left ankle", "right ankle"]

1/1 [==============================] - 0s 54ms/step
(56,)


In [88]:
max_key,key_val = points[0,:,55].argmax(), points[0,:,55].max()
max_key,key_val

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [83]:
keypoints_dict = {}
for i in range(0,len(points)-5,3):
    keypoints_dict[label[i//3]] = [points[i],points[i+1],points[i+2]]

In [84]:
keypoints_dict

{'nose': [0.0, 2496.6963, 2886.4797],
 'left eye': [0.0, 3126.6018, 0.0],
 'right eye': [9081.767, 0.0, 2753.1133],
 'left ear': [0.0, 10685.972, 0.0],
 'right ear': [7835.9697, 0.0, 2621.5486],
 'left shoulder': [6562.159, 0.0, 0.0],
 'right shoulder': [6568.4727, 397.92014, 5829.295],
 'left elbow': [388.74182, 2583.816, 3592.377],
 'right elbow': [2592.8137, 0.0, 3119.575],
 'left wrist': [0.0, 4311.229, 0.0],
 'right wrist': [0.0, 0.0, 0.0],
 'left hip': [4740.899, 9779.3, 12390.897],
 'right hip': [0.0, 0.0, 0.0],
 'left knee': [0.0, 755.8937, 1900.2125],
 'right knee': [0.0, 1595.8419, 9047.692],
 'left ankle': [2653.0066, 0.0, 7808.8193],
 'right ankle': [1061.5133, 4593.9614, 0.0]}